In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI

# Aufgabe 02: Minimax (einfacher Materialwert)

Dieses Notebook implementiert den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Die Evaluierung eines Boards wird dabei durch Betrachtung des reinen Materialwertes der Figuren realisiert.

Die Klasse wird um eine neue Konstante `DEPTH` erweitert. Diese gibt an, wie viele zukünftige Halbzüge bei der Berechnung des nächsten Zuges betrachtet werden sollen, das heißt, welche Tiefe der entsprechende Baum aller möglichen nächsten Halbzüge aufweist. Die Konstante wird als Parameter übergeben und im Konstruktor gesetzt.

Ferner wird die Variable `last_evaluation` hinzugefügt, die den Wert der letzten Evaluierung auf Instanz-Ebene speichert. Die Funktion `reset` dient dazu, die Instanz nach einem beendeten Spiel in ihren Ursprungszustand zurückzusetzen.

In [ ]:
import chess
import os


class Exercise02AI(ChessAI):
    """Chooses middle game moves using minimax algorithm and material values."""

    def __init__(self, search_depth: int = 3, **kwargs) -> None:
        super().__init__(**kwargs)
        self.DEPTH = search_depth
        self.last_evaluation: int | None = None

    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.last_evaluation = None

## Evaluierungsfunktion

Die Evaluierungsfunktion `evaluate` nimmt ein Board als Argument und berechnet eine Ganzzahl als Maß dafür, ob die gegebene Stellung für den weißen Spieler eher zu einem Sieg (positiver Wert) oder zu einer Niederlage (negativer Wert) führt.

Diese Implementierung verwendet dafür den einfachen Materialwert (Tauschwert in Bauerneinheiten) nach folgender Tabelle beschrieben in [8]:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 1  |
| Springer (knight) | 3  |
| Läufer (bishop) | 3  |
| Turm (rook) | 5  |
| Dame (queen) | 9  |

Für die Berechnung des nächsten Schrittes stehen mehrere Funktionen zur Verfügung:

- `full_evaluate`
- `incremental_evaluate`
- `evaluate`

Die Funktion `full_evaluate` berechnet den Wert eines Boards (`board`) von Grund auf neu und geht dabei wie folgt vor:

1. Es wird für jede Figur und für jede Farbe die Anzahl der Spielfiguren ermittelt.
2. Für jede Figur wird die Differenz der Anzahl pro Farbe gebildet ($\textrm{Weiß} - \textrm{Schwarz}$).
3. Jede Differenz wird mit dem figurenspezifischen Faktor multipliziert und zur Gesamtsumme addiert.

Die Funktion `incremental_evaluate` berechnet die Änderungen der Bewertung, wenn der gegebene Zug (`next_move`) auf das übergebene Board (`board`) angewendet wird. Diese Änderung wird zurückgegeben. Hierfür werden die drei Fälle untersucht in denen sich der Wert verändert haben kann:

1. Ein En Passant Zug wurde durchgeführt.
2. Eine Figur wurde geschlagen.
3. Ein Bauer wurde in eine neue Figur umgewandelt.

Die Funktion `evaluate` bekommt ein Board (`board`) und die Evaluierung dieses Boards *vor* dem letzten Halbzug (`last_evaluation`) und gibt die Summe der letzten Evaluierung und der berechneten Änderung für den letzten Zug zurück.

In [ ]:
class Exercise02AI(Exercise02AI):  # type: ignore
    MATERIAL_VALUES = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
    }

    def full_evaluate(self, board: chess.Board) -> int:
        """Returns a full evaluation of the given board."""
        evaluation = 0
        for piece_type in chess.PIECE_TYPES[:-1]:
            # Get the amount of pieces on the board per color
            amount_type_white = len(board.pieces(piece_type, chess.WHITE))
            amount_type_black = len(board.pieces(piece_type, chess.BLACK))
            # Get difference
            diff = amount_type_white - amount_type_black
            # Calculate material value
            evaluation += self.MATERIAL_VALUES[piece_type] * diff
        return evaluation

    def incremental_evaluate(
        self, board: chess.Board, last_evaluation: int, next_move: chess.Move
    ) -> int:
        """Returns an incrementally calculated evaluation of the given board."""
        change = 0

        if captured_piece_type := board.piece_type_at(next_move.to_square):
            change += self.MATERIAL_VALUES[captured_piece_type]

        if promotion_piece_type := next_move.promotion:
            change += (
                self.MATERIAL_VALUES[promotion_piece_type]
                - self.MATERIAL_VALUES[chess.PAWN]
            )

        elif captured_piece_type is None and board.is_en_passant(next_move):
            # If the first two cases didn't match, check for en passant move
            change += self.MATERIAL_VALUES[chess.PAWN]

        # Add to old evaluation if white to move, else subtract
        factor = +1 if board.turn else -1
        return last_evaluation + change * factor

## Minimax

Die rekursive `minimax`-Funktion betrachtet bei einem gegebenen Board alle möglichen Stellungen (Boards) nach einer gegebenen Anzahl von Halbzügen (Tiefe) und gibt die beste Evaluierung (`evaluate`-Funktion) für den aktuellen Spieler zurück. Hierfür bekommt die Funktion als Argumente das aktuelle Board (`board`), die Evaluierungstiefe in Halbzügen (`depth`) und die Evaluierung des übergebenen Boards (`current_evaluation`). Zurückgegeben wird ein Tupel bestehend aus der besten Evaluierung und des Halbzuges, der auf den Pfad zu dieser Evaluierung führt.

Die Konstante `LIMIT` wird als obere (`+`) beziehungsweise untere (`-`) Schranke für den Vergleich innerhalb der ersten Iteration verwendet.

Innerhalb der `minimax`-Funktion werden zwei Fälle unterschieden:

1. Falls Weiß am Zug ist, wird jeder mögliche Zug rekursiv evaluiert und das Maximum der Ergebnisse zurückgegeben.
2. Falls Schwarz am Zug ist, wird jeder mögliche Zug rekursiv evaluiert und das Minimum der Ergebnisse zurückgegeben.

### Hilfsfunktionen

Die Funktion `early_abort_evaluation` nimmt dieselben Argumente an, wie die zuvor beschriebene und wird zu Beginn der `minimax`-Funktion aufgerufen. Sie bricht mithilfe der folgenden Fallunterscheidung den Funktionsaufruf und damit die Rekursion ab:

1. Das Spiel ist ein Remis: In diesem Fall wird 0 als Wert zurückgegeben. Der Zug welcher zu diesem Zustand führt wird also nur gewählt, sofern jeder andere Zug zu einem für die Seite schlechteren Wert (und somit eher zu einer Niederlage) führt.
2. Weiß hat das Spiel gewonnen: Hier wird das positive Limit (`99999`) abzüglich der aktuellen Tiefe zurückgegeben, welches auf jeden Fall größer oder gleich dem Evaluierungswert aller anderen Züge ist. Weiß wird daher diesen (oder einen in dieser Hinsicht gleichwertigen) Zug wählen um das Spiel zu gewinnen, Schwarz versucht den Zug möglichst zu vermeiden.
3. Schwarz hat das Spiel gewonnen: Das negative Limit (`-99999`) abzüglich der aktuellen Tiefe wird zurückgegeben. Dieses ist auf jeden Fall kleiner oder gleich dem Evaluierungswert aller anderen Züge. Schwarz wird daher diesen (oder einen in dieser Hinsicht gleichwertigen) Zug wählen um das Spiel zu gewinnen, Weiß versucht den Zug möglichst zu vermeiden.
4. Zuletzt wird geprüft, ob die Tiefe Null ist. Auch dann wird die Rekursion beendet und die Auswertung des aktuellen Boards zurückgegeben.

In [ ]:
class Exercise02AI(Exercise02AI):  # type: ignore
    LIMIT = 99999

    def minimax_early_abort(
        self, board: chess.Board, depth: int, current_evaluation: int
    ) -> int | None:
        """Returns an evaluation iff the minimax has an early exit condition. Returns none otherwise."""
        if (is_checkmate := board.is_checkmate()) and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - (self.DEPTH - depth)
            return evaluation

        if is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + (self.DEPTH - depth)
            return evaluation

        if (
            board.is_insufficient_material()
            or not board.legal_moves
            or board.is_fifty_moves()
        ):
            # Game is a draw
            return 0

        # Recursion abort case
        if depth == 0:
            return current_evaluation

        return None

    def minimax(
        self, board: chess.Board, depth: int, current_evaluation: int
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm."""
        self.stats[-1]["nodes"] += 1
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            return early_abort_evaluation, None

        best_move = None

        # White to play (positive numbers are good)
        if board.turn:
            maxEvaluation = -self.LIMIT
            for move in board.legal_moves:
                new_evaluation = self.incremental_evaluate(
                    board, current_evaluation, move
                )
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, new_evaluation)
                board.pop()
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            minEvaluation = self.LIMIT
            for move in board.legal_moves:
                new_evaluation = self.incremental_evaluate(
                    board, current_evaluation, move
                )
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, new_evaluation)
                board.pop()
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

## Berechnung des besten Zuges

Die Funktion `get_next_middle_game_move` berechnet auf einem gegebenen Board den nächsten besten Zug. Dabei wird jeder mögliche Zug mithilfe der `minimax`-Funktion in der gewünschten Tiefe evaluiert und der Zug mit der besten Bewertung zurückgegeben. Zusätzlich wird die aktuelle Stellung evaluiert und das Ergebnis in der Variable `self.last_evaluation` gespeichert. Die Evaluierung erfolgt inkrementell:

- Hat die Variable `self.last_evaluation` der Wert `None`, so wird eine vollständige Evaluierung durchgeführt und das Ergebnis als `current_evaluation` gespeichert.
- Ist die letzte Evaluierung nicht `None`, so ist `current_evaluation` die Summe von `last_evaluation` und der Evaluierung des letzten gegnerischen Zuges.

Final wird die Bewertung des gewählten Zuges auf `current_evaluation` addiert.

Zuletzt werden in der Klassenvariable `self.stats` die aktuellen Spielmetriken gespeichert.

In [ ]:
class Exercise02AI(Exercise02AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.last_evaluation: int | None  # type annotation for mypy
        if self.last_evaluation is None:
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            last_move = board.pop()
            current_evaluation = self.incremental_evaluate(board, self.last_evaluation, last_move)  # type: ignore
            board.push(last_move)

        # Call minimax and get best move
        self.stats[-1]["nodes"] = 0
        future_evaluation, best_move = self.minimax(
            board, self.DEPTH, current_evaluation
        )
        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = self.incremental_evaluate(
            board, current_evaluation, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
# Create player and board
unit_test_player = Exercise02AI(player_name="Ex02AI", search_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test full evaluation
def test_full_evaluation(unit_test_player: ChessAI, Board: chess.Board):
    full_eval = unit_test_player.full_evaluate(board)
    print(f"Full Evaluation: {full_eval}")
    assert full_eval == 13, "Full evaluation does not match expected value!"

In [ ]:
test_full_evaluation(unit_test_player, board)

In [ ]:
# Test incremental evaluation
def test_incremental_evaluation(unit_test_player: ChessAI, board: chess.Board):
    next_move = chess.Move.from_uci("d4c3")  # white queen capture
    inc_eval = unit_test_player.incremental_evaluate(board, 0, next_move)
    print(f"Incremental Evaluation: {inc_eval}")
    assert inc_eval == -9, "Incremental evaluation does not match expected value!"

In [ ]:
test_incremental_evaluation(unit_test_player, board)

In [ ]:
# Test evaluation
def test_evaluation(unit_test_player: ChessAI, board: chess.Board):
    next_move = chess.Move.from_uci("d4c3")  # white queen capture
    full_eval, inc_eval = 13, -9
    evaluation = unit_test_player.incremental_evaluate(board, full_eval, next_move)
    board.push(next_move)
    new_full_eval = unit_test_player.full_evaluate(board)
    board.pop()
    print(f"Full Evaluation: {full_eval}")
    print(f"Incremental Evaluation: {inc_eval}")
    print(f"Evaluation: {evaluation}")
    print(f"New Full Evaluation: {new_full_eval}")
    assert new_full_eval == evaluation, "Incremental and full evaluation are different!"
    assert evaluation == full_eval + inc_eval, "Evaluation does not match expected sum!"

In [ ]:
test_evaluation(unit_test_player, board)

In [ ]:
# Test minimax
def test_minimax(
    unit_test_player: ChessAI,
    board: chess.Board,
    current_evaluation: int,
    expected_evaluation: int,
    expected_move: str,
    expected_nodes: int,
):
    unit_test_player.stats[-1]["nodes"] = 0
    mm_evaluation, mm_move = unit_test_player.minimax(
        board, unit_test_player.DEPTH, current_evaluation
    )
    nodes = unit_test_player.stats[-1]["nodes"]
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    print(f"Nodes searched: {nodes}")
    assert (
        mm_evaluation == expected_evaluation
    ), "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == expected_move, "Minimax move does not match expected value!"
    assert nodes == expected_nodes, "Searched node count has changed!"

In [ ]:
test_minimax(
    unit_test_player,
    board,
    current_evaluation=4,
    expected_evaluation=-5,
    expected_move="d4c3",
    expected_nodes=14377,
)

In [ ]:
# Test next move function
def test_next_move(
    unit_test_player: ChessAI,
    board: chess.Board,
    expected_move: str,
    expected_nodes: int,
):
    unit_test_player.stats[-1]["nodes"] = 0
    unit_test_player.last_evaluation = None
    move = unit_test_player.get_next_middle_game_move(board)
    nodes = unit_test_player.stats[-1]["nodes"]
    print(f"Move: {move}")
    print(f"Nodes searched: {nodes}")
    assert move.uci() == expected_move, "Next move does not match expected value!"
    assert nodes == expected_nodes, "Searched node count has changed!"

In [ ]:
test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=14377,
)

In [ ]:
# Test reset function
def test_reset(unit_test_player: ChessAI, _: chess.Board):
    unit_test_player.reset()
    assert unit_test_player.last_evaluation is None, "Reset was not successful!"

In [ ]:
test_reset(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.